# Import

In [1]:
import pandas as pd
import numpy as np
import csv
import math

# Cleaning data

CPI data only contains data from year 1975 to 2023, so we only import relevant data. 

In [2]:
data  = pd.DataFrame()
for year in range(1975, 2024):
    data = pd.concat([data, pd.read_csv('Brickset-Sets'+ str(year)+'.csv')])


Checking that all the data is in fact in one dataframe

In [3]:
print(data.Year)

0      1975
1      1975
2      1975
3      1975
4      1975
       ... 
919    2023
920    2023
921    2023
922    2023
923    2023
Name: Year, Length: 19137, dtype: int64


### Checking the price columns

In [4]:
print(data.columns)

Index(['Number', 'Theme', 'Subtheme', 'Year', 'Set name', 'Minifigs', 'Pieces',
       'RRP (GBP)', 'RRP (USD)', 'RRP (CAD)', 'RRP (EUR)', 'EAN', 'UPC',
       'Width', 'Height', 'Depth', 'Weight', 'Notes', 'Qty owned',
       'Qty owned new', 'Qty owned used', 'Wanted', 'Qty wanted', 'Priority',
       'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27',
       'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31',
       'Value new (USD)', 'Value used (USD)', 'Launch date', 'Exit date'],
      dtype='object')


We remove all RRP other than RRP(USD), we can also remove EAN and UPC, the two columns are barcode which is not useful in this project. 

In [5]:
data = data.drop(['RRP (GBP)', 'RRP (CAD)', 'RRP (EUR)', 'EAN', 'UPC'],axis=1)
print(data.columns)

Index(['Number', 'Theme', 'Subtheme', 'Year', 'Set name', 'Minifigs', 'Pieces',
       'RRP (USD)', 'Width', 'Height', 'Depth', 'Weight', 'Notes', 'Qty owned',
       'Qty owned new', 'Qty owned used', 'Wanted', 'Qty wanted', 'Priority',
       'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27',
       'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31',
       'Value new (USD)', 'Value used (USD)', 'Launch date', 'Exit date'],
      dtype='object')


Now we check the unamed columns

In [6]:
unnamed_columns = []
for i in range(24, 32):
    unnamed_columns.append('Unnamed: ' + str(i))
    print(data['Unnamed: ' + str(i)][:5])#making sure that the data are in fact NaN
    print(i, len(data['Unnamed: ' + str(i)]),np.isnan(data['Unnamed: '+ str(i)]).count())
print(unnamed_columns)

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: Unnamed: 24, dtype: float64
24 19137 19137
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: Unnamed: 25, dtype: float64
25 19137 19137
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: Unnamed: 26, dtype: float64
26 19137 19137
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: Unnamed: 27, dtype: float64
27 19137 19137
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: Unnamed: 28, dtype: float64
28 19137 19137
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: Unnamed: 29, dtype: float64
29 19137 19137
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: Unnamed: 30, dtype: float64
30 19137 19137
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: Unnamed: 31, dtype: float64
31 19137 19137
['Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31']


In those unnames columns, there is no values associated, it is safe to remove those columns

In [7]:
data.drop(columns=unnamed_columns, inplace = True)
print(data.columns)

Index(['Number', 'Theme', 'Subtheme', 'Year', 'Set name', 'Minifigs', 'Pieces',
       'RRP (USD)', 'Width', 'Height', 'Depth', 'Weight', 'Notes', 'Qty owned',
       'Qty owned new', 'Qty owned used', 'Wanted', 'Qty wanted', 'Priority',
       'Value new (USD)', 'Value used (USD)', 'Launch date', 'Exit date'],
      dtype='object')


In [8]:
for col in data.columns:
    print(data[col].dtype)

object
object
object
int64
object
float64
float64
float64
float64
float64
float64
float64
float64
int64
int64
int64
float64
int64
int64
float64
float64
object
object


All of the columns have a data type object, we now need to convert each column's data into appropriate data type. 

Checking the type of data in uncertain columns

In [9]:
for col in data.columns:
    print(col)
    print(data[col][pd.notna(data[col])][:5])

Number
0     75-1
1     77-1
2    077-1
3     78-1
4     78-3
Name: Number, dtype: object
Theme
0    PreSchool
1    PreSchool
2        Duplo
3    PreSchool
4    Samsonite
Name: Theme, dtype: object
Subtheme
4    Basic set
5      4.5/12V
6      4.5/12V
7      4.5/12V
8      4.5/12V
Name: Subtheme, dtype: object
Year
0    1975
1    1975
2    1975
3    1975
4    1975
Name: Year, dtype: int64
Set name
0     PreSchool Set
1     PreSchool Set
2    Pre-School Set
3     PreSchool Set
4         Basic Set
Name: Set name, dtype: object
Minifigs
9     5.0
17    2.0
18    7.0
19    7.0
20    8.0
Name: Minifigs, dtype: float64
Pieces
0     16.0
1     20.0
2     21.0
3     32.0
4    330.0
Name: Pieces, dtype: float64
RRP (USD)
13     1.00
35    14.95
0      2.75
47    15.00
56     5.00
Name: RRP (USD), dtype: float64
Width
40     15.0
106    10.5
124    19.2
111     9.5
112     9.5
Name: Width, dtype: float64
Height
40     6.5
106    7.0
124    9.5
111    5.8
112    5.8
Name: Height, dtype: float64
D

From the above ouput: Notes column are empty, 

In [10]:
data.drop(columns='Notes', inplace=True)
print(data.columns)

Index(['Number', 'Theme', 'Subtheme', 'Year', 'Set name', 'Minifigs', 'Pieces',
       'RRP (USD)', 'Width', 'Height', 'Depth', 'Weight', 'Qty owned',
       'Qty owned new', 'Qty owned used', 'Wanted', 'Qty wanted', 'Priority',
       'Value new (USD)', 'Value used (USD)', 'Launch date', 'Exit date'],
      dtype='object')


In [11]:
data['Number'] = data['Number'].astype('string')
data['Theme'] = data['Theme'].astype('string')
data['Subtheme'] = data['Subtheme'].astype('string')
data['Set name'] = data['Set name'].astype('string')
data['Launch date'] = pd.to_datetime(data['Launch date'],format='%d/%m/%Y')
data['Exit date'] = pd.to_datetime(data['Exit date'],format='%d/%m/%Y')

In [12]:
for col in data.columns:
    print(data[col].dtype)

string
string
string
int64
string
float64
float64
float64
float64
float64
float64
float64
int64
int64
int64
float64
int64
int64
float64
float64
datetime64[ns]
datetime64[ns]


In [13]:
print(data.columns)

Index(['Number', 'Theme', 'Subtheme', 'Year', 'Set name', 'Minifigs', 'Pieces',
       'RRP (USD)', 'Width', 'Height', 'Depth', 'Weight', 'Qty owned',
       'Qty owned new', 'Qty owned used', 'Wanted', 'Qty wanted', 'Priority',
       'Value new (USD)', 'Value used (USD)', 'Launch date', 'Exit date'],
      dtype='object')


In [14]:
df = data[['Number', 'Theme', 'Subtheme', 'Year', 'Set name', 'Minifigs', 'Pieces',
       'RRP (USD)', 'Value new (USD)', 'Value used (USD)', 'Launch date', 'Exit date']]
df.head

<bound method NDFrame.head of          Number               Theme                Subtheme  Year  \
0          75-1           PreSchool                    <NA>  1975   
1          77-1           PreSchool                    <NA>  1975   
2         077-1               Duplo                    <NA>  1975   
3          78-1           PreSchool                    <NA>  1975   
4          78-3           Samsonite               Basic set  1975   
..          ...                 ...                     ...   ...   
919  SDCC2023-5                Gear                    Pins  2023   
920  SDCC2023-6                Gear           Miscellaneous  2023   
921  SDCC2023-7                Gear           Miscellaneous  2023   
922  SDCC2023-9                Gear  Key Chains/Promotional  2023   
923   SONICIK-1  Sonic the Hedgehog             Promotional  2023   

                                   Set name  Minifigs  Pieces  RRP (USD)  \
0                             PreSchool Set       NaN    16.0    

In [15]:
print(df['RRP (USD)'].isna().sum())
df.dropna(subset=['RRP (USD)'], inplace = True)
print(df['RRP (USD)'].isna().sum())

7391
0


/var/folders/5g/q6jfbkkj3rz2mb5hlkd06mkh0000gq/T/ipykernel_4694/1026020644.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=['RRP (USD)'], inplace = True)


In [16]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 11746 entries, 13 to 901
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Number            11746 non-null  string        
 1   Theme             11746 non-null  string        
 2   Subtheme          9869 non-null   string        
 3   Year              11746 non-null  int64         
 4   Set name          11746 non-null  string        
 5   Minifigs          6493 non-null   float64       
 6   Pieces            9654 non-null   float64       
 7   RRP (USD)         11746 non-null  float64       
 8   Value new (USD)   9138 non-null   float64       
 9   Value used (USD)  8368 non-null   float64       
 10  Launch date       6621 non-null   datetime64[ns]
 11  Exit date         6621 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(5), int64(1), string(4)
memory usage: 1.2 MB
None


In [17]:
complete_lego  = df.to_csv('complete_lego_price.csv')
